# <center> Quantitative Value Strategy

## Library importe

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore as score
from statistics import mean
import math

## Importing a list of stocks & API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making the fisrt API call

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 94489726,
 'calculationPrice': 'close',
 'change': -2.06,
 'changePercent': -0.01229,
 'close': 170.52,
 'closeSource': 'cffliiao',
 'closeTime': 1690250349106,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 177.262,
 'delayedPriceTime': 1688527583264,
 'extendedChange': -0.3,
 'extendedChangePercent': -0.00182,
 'extendedPrice': 170.82,
 'extendedPriceTime': 1700557616934,
 'high': 180.24,
 'highSource': ' arEercitIpil eeX m',
 'highTime': 1720787055664,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 174.16,
 'iexCloseTime': 1670404470381,
 'iexLastUpdated': 1731734652435,
 'iexMarketPercent': 0.019015715869592317,
 'iexOpen': 178.65,
 'iexOpenTime': 1663872670772,
 'iexRealtimePrice': 170.76,
 'iexRealtimeSize': 100,
 'iexVolume': 1503112,
 'lastTradeTime': 1666358854590,
 'latestPrice': 171.16,
 'latestSource': 'Close',
 'latestTime': 'April 8, 2022',
 'latestUpdate': 1667543610792,
 'latestVolume': 798851

## Parsing the API call

In [4]:
price = data['latestPrice']
pe_retio = data['peRatio']

## Executing a Batch API call and Building a DataFrame

In [5]:
# funtion sourced from

def chunks(lst, n):
    """ Yield succesive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price to Earning Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series(
           [
               symbol,
               data[symbol]['quote']['latestPrice'],
               data[symbol]['quote']['peRatio'],
               'N/A'
           ], index=my_columns),
        ignore_index=True
        )
final_dataframe

,Ticker,Price,Price to Earning Ratio,Number of Shares to Buy
0,A,139.88,35.86,N/A
1,AAL,16.68,-5.42,N/A
2,AAP,233.17,24.01,N/A
3,AAPL,177.95,28.44,N/A
4,ABBV,176.89,27.86,N/A
5,ABC,167.35,21.51,N/A
6,ABMD,314.76,113.32,N/A
7,ABT,128.44,31.7,N/A
8,ACN,339.94,34.2,N/A
9,ADBE,466.76,45.14,N/A


## Removing Glamour Stocks

In [7]:
final_dataframe.sort_values('Price to Earning Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price to Earning Ratio']>0]
final_dataframe = final_dataframe[:50].reset_index()
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe


,Ticker,Price,Price to Earning Ratio,Number of Shares to Buy
0,EBAY,57.47,2.73,N/A
1,F,15.77,3.45,N/A
2,BIO,618.30,4.35,N/A
3,NRG,40.63,4.5,N/A
4,COF,137.92,4.91,N/A
5,SYF,38.21,5.1,N/A
6,C,50.94,5.26,N/A
7,GS,329.16,5.45,N/A
8,PVH,75.63,5.6,N/A
9,PHM,41.48,5.64,N/A


## Calculating the Number of Shares to Buy

In [8]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = float(input('Enter the value of your portfolio: '))
            break
        except ValueError:
                    print("Thats not  number! \nPlease try again")

In [9]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [10]:
position_size = portfolio_size / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price to Earning Ratio,Number of Shares to Buy
0,EBAY,57.47,2.73,870
1,F,15.77,3.45,3170
2,BIO,618.30,4.35,80
3,NRG,40.63,4.5,1230
4,COF,137.92,4.91,362
5,SYF,38.21,5.1,1308
6,C,50.94,5.26,981
7,GS,329.16,5.45,151
8,PVH,75.63,5.6,661
9,PHM,41.48,5.64,1205


## Building a Better and more Realistic Value Strategy

In [28]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price to earning retio
pe_ratio = data[symbol]['quote']['peRatio']

# Price to book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price to sale ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earning Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [29]:
# rv stand for robust value
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price to earning retio',
    'PE Percentile',
    'Price to book ratio',
    'PB Percentile',
    'Price to sale ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_ebitda = np.NaN
            ev_to_gross_profit =  np.NaN
        rv_dataframe = rv_dataframe.append(pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ], index=rv_columns),
        ignore_index = True
        )
rv_dataframe    

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.13,N/A,34.9,N/A,8.09,N/A,6.37,N/A,24.410092,N/A,11.849543,N/A,N/A
1,AAL,17.18,N/A,-5.51,N/A,-1.5,N/A,0.3795,N/A,-12.507893,N/A,1.231017,N/A,N/A
2,AAP,228.07,N/A,24.49,N/A,4.28,N/A,1.27,N/A,12.100427,N/A,2.794775,N/A,N/A
3,AAPL,171.55,N/A,29.25,N/A,39.38,N/A,7.56,N/A,22.086403,N/A,17.293219,N/A,N/A
4,ABBV,176.59,N/A,28.04,N/A,20.53,N/A,5.69,N/A,13.250673,N/A,9.786117,N/A,N/A
5,ABC,167.66,N/A,22.06,N/A,145.27,N/A,0.1608,N/A,10.650753,N/A,5.031815,N/A,N/A
6,ABMD,318.45,N/A,112.95,N/A,10.36,N/A,14.53,N/A,50.690272,N/A,17.093324,N/A,N/A
7,ABT,126.04,N/A,31.7,N/A,6.25,N/A,5.1,N/A,21.475258,N/A,9.276953,N/A,N/A
8,ACN,345.84,N/A,34.5,N/A,11.51,N/A,4.14,N/A,20.352578,N/A,11.830745,N/A,N/A
9,ADBE,454.53,N/A,45.11,N/A,15.64,N/A,13.47,N/A,34.684859,N/A,14.849641,N/A,N/A


## Dealing With the Missing Data in the DataFrame

In [30]:
# Return DataFrame with all the missing data
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.900,N/A,61.14,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,338.250,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,367.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.060,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.420,N/A,14.36,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,60.170,N/A,33.52,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,37.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,127.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
203,GOOG,2743.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [31]:
# Return the number of row with missing data
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)])

19

In [32]:
missing_values_columns = ['Price to earning retio','Price to book ratio', 'Price to sale ratio', 'EV/EBITDA', 'EV/GP', ]

In [33]:
for column in missing_values_columns:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

## Calculating Value Percentile

In [34]:
metrics = {
    'Price to earning retio': 'PE Percentile',
    'Price to book ratio':'PB Percentile',
    'Price to sale ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.13,N/A,34.900000,0.736527,8.090000,0.794411,6.37000,0.808383,24.410092,0.804391,11.849543,0.786427,N/A
1,AAL,17.18,N/A,-5.510000,0.0399202,-1.500000,0.0598802,0.37950,0.0179641,-12.507893,0.0518962,1.231017,0.0419162,N/A
2,AAP,228.07,N/A,24.490000,0.508982,4.280000,0.597804,1.27000,0.164671,12.100427,0.397206,2.794775,0.143713,N/A
3,AAPL,171.55,N/A,29.250000,0.62475,39.380000,0.974052,7.56000,0.852295,22.086403,0.762475,17.293219,0.898204,N/A
4,ABBV,176.59,N/A,28.040000,0.60479,20.530000,0.934132,5.69000,0.767465,13.250673,0.45509,9.786117,0.686627,N/A
5,ABC,167.66,N/A,22.060000,0.46507,145.270000,0.998004,0.16080,0.00399202,10.650753,0.341317,5.031815,0.305389,N/A
6,ABMD,318.45,N/A,112.950000,0.974052,10.360000,0.828343,14.53000,0.946108,50.690272,0.974052,17.093324,0.896208,N/A
7,ABT,126.04,N/A,31.700000,0.670659,6.250000,0.722555,5.10000,0.722555,21.475258,0.732535,9.276953,0.658683,N/A
8,ACN,345.84,N/A,34.500000,0.722555,11.510000,0.848303,4.14000,0.60479,20.352578,0.718563,11.830745,0.784431,N/A
9,ADBE,454.53,N/A,45.110000,0.834331,15.640000,0.914172,13.47000,0.936128,34.684859,0.93014,14.849641,0.858283,N/A


## Calculating the RV Score

In [35]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,141.13,N/A,34.900000,0.736527,8.090000,0.794411,6.37000,0.808383,24.410092,0.804391,11.849543,0.786427,0.786028
1,AAL,17.18,N/A,-5.510000,0.0399202,-1.500000,0.0598802,0.37950,0.0179641,-12.507893,0.0518962,1.231017,0.0419162,0.0423154
2,AAP,228.07,N/A,24.490000,0.508982,4.280000,0.597804,1.27000,0.164671,12.100427,0.397206,2.794775,0.143713,0.362475
3,AAPL,171.55,N/A,29.250000,0.62475,39.380000,0.974052,7.56000,0.852295,22.086403,0.762475,17.293219,0.898204,0.822355
4,ABBV,176.59,N/A,28.040000,0.60479,20.530000,0.934132,5.69000,0.767465,13.250673,0.45509,9.786117,0.686627,0.689621
5,ABC,167.66,N/A,22.060000,0.46507,145.270000,0.998004,0.16080,0.00399202,10.650753,0.341317,5.031815,0.305389,0.422754
6,ABMD,318.45,N/A,112.950000,0.974052,10.360000,0.828343,14.53000,0.946108,50.690272,0.974052,17.093324,0.896208,0.923752
7,ABT,126.04,N/A,31.700000,0.670659,6.250000,0.722555,5.10000,0.722555,21.475258,0.732535,9.276953,0.658683,0.701397
8,ACN,345.84,N/A,34.500000,0.722555,11.510000,0.848303,4.14000,0.60479,20.352578,0.718563,11.830745,0.784431,0.735729
9,ADBE,454.53,N/A,45.110000,0.834331,15.640000,0.914172,13.47000,0.936128,34.684859,0.93014,14.849641,0.858283,0.894611


## Selecting the 50 best Value Stocks 

In [36]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.18,N/A,-5.51,0.0399202,-1.5000,0.0598802,0.3795,0.0179641,-12.507893,0.0518962,1.231017,0.0419162,0.0423154
1,UNM,33.37,N/A,8.38,0.117764,0.5617,0.0638723,0.5353,0.0439122,3.794406,0.0798403,0.522772,0.00199601,0.061477
2,PRU,122.46,N/A,6.02,0.0818363,0.7118,0.0718563,0.6085,0.0598802,3.980579,0.0858283,0.606727,0.00798403,0.061477
3,PVH,73.29,N/A,5.70,0.0698603,0.9316,0.0858283,0.5482,0.0459082,4.947698,0.113772,1.122495,0.0299401,0.0690619
4,GM,39.57,N/A,6.14,0.0858283,0.9860,0.0918164,0.4640,0.0359281,1.897589,0.0598802,1.610759,0.0758483,0.0698603
5,AIG,65.88,N/A,5.93,0.0758483,0.7930,0.0738523,0.9580,0.111776,3.653036,0.0738523,0.944167,0.0199601,0.0710579
6,LNC,66.20,N/A,8.75,0.125749,0.5405,0.0618762,0.5875,0.0578842,6.149280,0.149701,0.552839,0.00399202,0.0798403
7,C,52.97,N/A,5.27,0.0638723,0.5659,0.0658683,1.2700,0.164671,3.252416,0.0678643,1.216541,0.0399202,0.0804391
8,F,15.64,N/A,3.44,0.0538922,1.2500,0.169661,0.4345,0.0299401,3.753910,0.0778443,1.996590,0.0998004,0.0862275
9,MET,71.17,N/A,9.99,0.149701,0.8551,0.0798403,0.8272,0.0938124,5.458461,0.131737,0.813418,0.0139721,0.0938124


## Calculating the Number od Shares to Buy

In [37]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [38]:
position_size = portfolio_size/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price to earning retio,PE Percentile,Price to book ratio,PB Percentile,Price to sale ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.18,2910,-5.51,0.0399202,-1.5000,0.0598802,0.3795,0.0179641,-12.507893,0.0518962,1.231017,0.0419162,0.0423154
1,UNM,33.37,1498,8.38,0.117764,0.5617,0.0638723,0.5353,0.0439122,3.794406,0.0798403,0.522772,0.00199601,0.061477
2,PRU,122.46,408,6.02,0.0818363,0.7118,0.0718563,0.6085,0.0598802,3.980579,0.0858283,0.606727,0.00798403,0.061477
3,PVH,73.29,682,5.70,0.0698603,0.9316,0.0858283,0.5482,0.0459082,4.947698,0.113772,1.122495,0.0299401,0.0690619
4,GM,39.57,1263,6.14,0.0858283,0.9860,0.0918164,0.4640,0.0359281,1.897589,0.0598802,1.610759,0.0758483,0.0698603
5,AIG,65.88,758,5.93,0.0758483,0.7930,0.0738523,0.9580,0.111776,3.653036,0.0738523,0.944167,0.0199601,0.0710579
6,LNC,66.20,755,8.75,0.125749,0.5405,0.0618762,0.5875,0.0578842,6.149280,0.149701,0.552839,0.00399202,0.0798403
7,C,52.97,943,5.27,0.0638723,0.5659,0.0658683,1.2700,0.164671,3.252416,0.0678643,1.216541,0.0399202,0.0804391
8,F,15.64,3196,3.44,0.0538922,1.2500,0.169661,0.4345,0.0299401,3.753910,0.0778443,1.996590,0.0998004,0.0862275
9,MET,71.17,702,9.99,0.149701,0.8551,0.0798403,0.8272,0.0938124,5.458461,0.131737,0.813418,0.0139721,0.0938124


## Formating the Excel Output

In [39]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Format Needed for the .xlsx File

In [40]:
background_color = '#0a023'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
float_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [41]:
column_formats = {
                 'A':['Ticker', string_format],
                 'B':['Price', dollar_format],
                 'C':['Number of Shares to Buy', integer_format],
                 'D':['Price to earning retio', float_format],
                 'E':['PE Percentile', percent_format],
                 'F':['Price to book ratio', float_format],
                 'G':['PB Percentile', percent_format],
                 'H':['Price to sale ratio', float_format],
                 'I':['PS Percentile', percent_format],
                 'J':['EV/EBITDA', float_format],
                 'K':['EV/EBITDA Percentile', percent_format],
                 'L':['EV/GP', float_format],
                 'M':['EV/GP Percentile', percent_format],
                 'N':['RV Score, string_format', percent_format]
                }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Excel Output

In [42]:
writer.save()